# Setup and data cleaning

In [118]:
# Dependencies
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [119]:
# Import IMDB movie data
title_basics = pd.read_csv('../data/title.basics.tsv',delimiter='\t',encoding='utf-8-sig',low_memory=False)

# Import ratings
title_ratings = pd.read_csv('../data/title.ratings.tsv',delimiter='\t',encoding='utf-8-sig')

# Import crew
title_crew = pd.read_csv('../data/title.crew.tsv',delimiter='\t',encoding='utf-8-sig')

# Import name basics
name_basics = pd.read_csv('../data/name.basics.tsv',delimiter='\t',encoding='utf-8-sig')

# Import box office data from BoxOfficeMojo
box_office = pd.read_csv('../data/boxoffice.csv')

# Import Oscar data from Wikipedia: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films
oscars = pd.read_csv('../data/oscars_cleaned.csv')

In [120]:
# Filter non-movies, adult movies
title_filtered = title_basics[title_basics['titleType']=='movie']
title_filtered = title_filtered[title_filtered['isAdult']==0]

In [129]:
# Split out genres and join
genres_split = title_filtered["genres"].str.split(",", n=2, expand=True)
joined = title_filtered.join(genres_split)

In [131]:
# Rename and drop some columns
cleaned = joined.rename(columns = {'tconst':'IMDB ID', 'titleType': 'Type', 'primaryTitle': 'Title', 'originalTitle': 'Title (original)', 'startYear': 'Year', 'runtimeMinutes': 'Runtime (min)', 0:'Genre (main)', 1:'Genre (sub 1)', 2:'Genre (sub 2)'})
cleaned = cleaned.drop(columns=['endYear', 'genres', 'Genre (sub 1)', 'Genre (sub 2)', 'Title (original)'])

In [137]:
# Drop empty rows
cleaned = cleaned.dropna(axis='rows')

,IMDB ID,Type,Title,isAdult,Year,Runtime (min),Genre (main)
8,tt0000009,movie,Miss Jerry,0,1894,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,20,Documentary
332,tt0000335,movie,Soldiers of the Cross,0,1900,\N,Biography
499,tt0000502,movie,Bohemios,0,1905,100,\N
571,tt0000574,movie,The Story of the Kelly Gang,0,1906,70,Biography


In [140]:
# Merge basic set and rating
with_ratings = cleaned.set_index('IMDB ID').join(title_ratings.set_index('tconst'))
with_ratings = with_ratings.rename(columns = {'averageRating': 'Rating (avg.)', 'numVotes': 'Votes'})

# Merge box office and Oscars
merged = pd.merge(box_office, oscars, left_on='title', right_on='Film', how='outer')

In [263]:
# Merge both of above to make combined set
combined = with_ratings.merge(merged,how='left', left_on='Title', right_on='title')

In [265]:
combined.head()

,Type,Title,isAdult,Year_x,Runtime (min),Genre (main),Rating (avg.),Votes,rank,title,studio,lifetime_gross,year,Film,Year_y,Awards,Nominations
0,movie,Miss Jerry,0,1894,45,Romance,5.9,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,movie,The Corbett-Fitzsimmons Fight,0,1897,20,Documentary,5.2,343.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,movie,Soldiers of the Cross,0,1900,\N,Biography,6.1,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,movie,Bohemios,0,1905,100,\N,4.4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,movie,The Story of the Kelly Gang,0,1906,70,Biography,6.1,570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
# Drop, rename
combined = combined.drop(['Type', 'isAdult', 'Year_y', 'year', 'rank', 'title', 'Film'], axis=1)
combined = combined.rename(columns = {'Year_x': 'Year', 'studio': 'Studio', 'lifetime_gross': 'Gross (lifetime)'})

In [273]:
combined.count()

Title               540781
Year                540781
Runtime (min)       540781
Genre (main)        540781
Rating (avg.)       243681
Votes               243681
Studio               27293
Gross (lifetime)     27357
Awards                1312
Nominations           1312
dtype: int64

## Things we could integrate

In [ ]:
# https://towardsdatascience.com/collecting-movie-data-445ca1ead8e5
# Budget
# Country (e.g. most popular genres by country)

# Calculate avg gross ($) per movie
#box_plot['Avg. lifetime gross'] = box_plot['lifetime_gross']/box_plot['title']

# Reset index to get year in column
#box_plot.reset_index()

# Machine Learning

In [ ]:
# - [prediction] 1/04-Stu_Brains
# - [multiple vectors] 1/05-Ins_Multiple_Linear_Regression_Sklearn
# - 1/06-Stu_Beer_Foam